In [95]:
from googleapiclient.discovery import build
import config
from datetime import datetime
import json
import os
import pandas as pd

In [2]:
KIDS = config.KIDS

In [40]:
class LinkCollector(object):
    
    '''
    This class used to search for related links of certain key words from youtube
    '''
    def __init__(self, query_key_word):

        self.query_key_word = query_key_word
        self.df_list = []
        self.resource = build('youtube', 'v3', developerKey=KIDS)
        self.cur_res = '1'
        self.res = []

    def search(self, next_page_token = None):
        
        cur_page_token = next_page_token
        import copy
        req = self.resource.search().list(part='snippet',
                                          maxResults = 50,
                                          order = 'relevance',
                                          pageToken = next_page_token,
                                          q=self.query_key_word,
                                          relevanceLanguage = 'en',
                                          type = 'video',
                                          videoCaption = 'any'
                                          )
        res = req.execute()
        self.cur_res = res
        self.res.append(copy.deepcopy(self.cur_res))
        self.__persist_data(res, cur_page_token)

    def __generate_datetime(self, year, month, day):

        return datetime(year=year, month=month, day = day).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    def __persist_data(self, res, cur_page_token):
        if cur_page_token is None:
            cur_page_token = ''
        json.dump(res, open('youtube_request_' + cur_page_token +'.json' , 'a+'))
        

In [66]:
def extract_video_from_json(directory):
    
    ### extract all the videos ids from the saved json file in a directory
    videos = []
    for entry in os.scandir(directory):
        if entry.path.endswith(".json") and entry.is_file():
            items = json.load(open(entry, 'r'))
            new_videos = [item['id']['videoId'] for item in items['items']]
            videos.extend(new_videos)

    return videos

In [67]:
videos = extract_video_from_json('/Users/QiaoLiu1/kids_tube/collected_link_json')

In [63]:
class VideoDetailsCollector(object):

    '''
    This class is used to search for detailed information of each videos from youtube
    '''
    def __init__(self, video_list):
        self.video_list = video_list
        self.items = []
        self.resource = build('youtube', 'v3', developerKey=KIDS)

    def retrieve(self):

        for i in range(0, len(self.video_list), 50):
            query_video = ','.join(self.video_list[i:i+50])
            req = self.resource.videos().list(part = 'snippet,contentDetails,statistics,topicDetails',
                                              maxResults = 50,
                                              id = query_video,
                                              )

            import copy
            res = req.execute()
            self.items.append(copy.deepcopy(res))
            if res is not None:
                self.__persist_data(res, self.video_list[i] + '_' + str(i))

    def __generate_datetime(self, year, month, day):
        return datetime(year=year, month=month, day=day).strftime('%Y-%m-%dT%H:%M:%SZ')

    def __persist_data(self, res, unique_identifier):
        if unique_identifier is None:
            unique_identifier = ''
        json.dump(res, open('youtube_request_' + unique_identifier + '.json', 'a+'))

In [70]:
video_details = VideoDetailsCollector(video_list=videos)

In [71]:
video_details.retrieve()

In [102]:
def extract_video_tag_des(directory):
    
    '''
    extract video information from json to data frame
    '''

    videos_df = pd.DataFrame(columns = ['video', 'title', 'tags', 'description'])

    for entry in os.scandir(directory):
        if entry.path.endswith(".json") and entry.is_file():
            items = json.load(open(entry, 'r'))
            video_ids = [item['id'] for item in items['items']]
            video_titles = [item['snippet']['title'] for item in items['items']]
            video_tags = [','.join(item['snippet']['tags']) for item in items['items']]
            video_desc = [item['snippet']['description'] for item in items['items']]
            videos_df = pd.concat([videos_df, pd.DataFrame({'video': video_ids,
                                                     'title': video_titles,
                                                     'tags': video_tags,
                                                     'description': video_desc})])

    return videos_df

In [237]:
import io 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/QiaoLiu1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/QiaoLiu1/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [258]:
def remove_stop(text):
    print(text)
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    return tokens_without_sw

In [239]:
res_df = pd.read_csv('./toys_video_details.csv', index_col = 0)

In [ ]:
res_df['tags_w/o_stop'] = res_df.tags.apply(remove_stop)

balloons toys,balloons,toys for kids,toys,family fun,kids playing,wendy,toys and colors,pretend play,kids pretend play,kids toys,fun,play,children,egg surprise,surprise eggs,huge eggs
car toys,car toys kids,cars for children,cars for kids,toys car,car kids,kids toys,kids video,toys for kids,fun,transformer toy,Toy For Kids,토이포키즈
toy,toys,pretend play,pretend,play,magic,magic toys,kids play,kids pretend play,magic story,magic stories,stories,kids video,toys for kids,for kids,children playing,kids toys
bombeiros,carrinhos,para crianças,fire truck,toys,toy vehicles,for kids,fire trucks,fire engine,fire,truck,trucks,toy,vehicles,fireman,kids
[]
toy,toys,kids toys,toys for kids,whoopee cushion,whoopie cushion,giant whoopee cushion,toy challenge,children activities,fun games,kids challenge,kids video,kid friendly,Ryan ToysReview
love Diana toys,love diana,love diana dolls,diana and roma,love Diana mystery shopper,kids diana show,diana roma,pretend play,princess,pop star,doll,mystery shopper,

tool toys,toolbox toys,kids tools,chocolate,candy,toy,toys,kids play,kids pretend play,pretend play,for kids,kids food,kids chocolate,candy chocolate,funny stories,kids video,children,toddlers,candy haul
[]
toys,clean up,cleaning toys,kids playing,mess,messy kids,playing,toys and colors,video for kids,children,pretend play,kids pretend play
Toy,Toys,Kids,Kid,Educational,Teach kids,Teach toddler,Preschool toys,Nick Jr,Paw Patrol,Peppa Pig,Paw Patrol English,Paw Patrol toy,Paw Patrol video,Peppa pig toy,peppa pig english,peppa pig video,educational toys,wooden toys,play,kid friendly,family friendly,pup,pig,preschool video,for kids,for children,for toddlers,surprises
kids video,kids car,kids cycle,kids toys,kids bike pc poyda kunalsinh Prakashsinh Chauhan jaynilsinh dineshsinh chauhan nana poyda
mcdonalds,mcdonalds toys,pretend play,pretend play mcdonalds,Ryan ToysReview,Ryan's Toy,Ryan Toys,toys,kids toys,kitchen toys,cooking toys,pretend food,pretend food toys,kitchen playset,hamburger 

Toys,Kids,Kids Toys,Surprise,Giant Egg,Egg Giant,Surprise Eggs,Surprise Egg,Play-Doh,Play-Doh Eggs,Playdoh Egg,Giant Surprise,Surprise Toys,Avengers,Avengers Toys,Marvel Toys,Iron Man,Captain America,Spiderman,Hulk,Spiderman Toys,Action Figures,Kids Cars,Marvel Avengers,Marvel Superheroes,Superhero Eggs,Avengers Eggs,Kids Playing,Kinder Egg,Kinder Egg Surprise,Family Fun,costume runway,costume,plastillina,halloween,irl,in real life
toys,kids toys,fraidy cats,board game,family game night,family,family fun,dogs,cats,kids game,Ryan ToysReview,Ryan's Family Review,surprise,surprise toys,toys surprise,kids surprise,kids playing game,egg surprise,children,toddlers,activities,kids video,kids channel,kids toy channel,family game
toy,doll,review,education,roleplay,play,demonstrate,pretend,create,unboxing,new,channel,story,videos,for,kids,make,demo,demonstration,children,toddler,preschool,boy,girl,teen,tween,fun,great,best,good,top,show,design,build,friendly,safe,game,entertainment,cute,lev,lego

doll,for kids,baby doll,baby doll toy,doll toys,toy,toys,toys and colors,pretend play,kids pretend play,emma,emma pretend play,toys for kids,kids playing,kids play,kids toys,play with dolls,play,kids and dolls,kids activities,story for kids,toys play
color,colors,kids colors,pretend play,kids pretend play,toy,toys,toy colors,color toy,color toys,children playing,kids play,kids playing,kids,play,fun,toys and colors,family fun,magic colors,playing,learning colors,play with colors,stories for kids,kids videos,play toys,new stories for kids,for kids
[]
peppa,peppa pig,toy video,educational,for kids,toy learning,best,preschool,videos for kids,videos for toddlers,toddlers,teach kids,colors,words,new house,house,playhouse,learn colors,toddler learning video,best learning videos,kids videos
[]
[]
[]
diana,diana and roma,toys,dolls,kindi kids,kindi kids toys,moose toys,kids toys,kids diana show,kids videos
[]
[]
[]
Lucy Dog,Puppy,Puppy Suprise,Lucy,Dog,Dogs,Puppies,Babies,Surprises,Toys,Plush,R

kids,play,mommy,pretend,funny,children,playtime,Naiah and Elli
DisneyCarToys,Barbie,Anna,Play-Doh,Play Doh Ice Cube,Playdough,Frozen Play Doh,Play Doh Videos,Play Doh Video,Olaf,Sven,Playdoh,Play Doh Playset,DisneyCarToys Frozen,DisneyCarToys Play Doh,Toys,Play-Doh Video,Play-Doh Surprise
practice,game,design,safe,demonstration,roleplay,woods,lev,friendly,for,target,demonstrate,cute,girl,children,new,make,lego,teen,tween,videos,fun,set,channel,best,adventure,boy,pretend,2016,show,story,good,play,heartlake,city,lego friends,hedgehog,gold,doll,apple,toddler,review,toy,preschool,shoot,top,camp,education,create,kids,great,speed,demo,trophy,archery,entertainment,mia,build,unboxing,forest
Plants Vs Zombies,plants vs. Zombies,Garden Warfare,Plants Vs Zombies plush,Plants Vs Zombies plush toys,plushie video,kids plushie video,Ryan ToysReview,Ryantoysreview,Ryan's Pretend play,pretend play for kids,plush toys for kids,in real life,toys in real life,Ryan's Toy Review,Ryan's Toy
Cedar Summit,Ston

...

[]
Learn,Colors,Fishing,Playset,fish,for Kids,for children,for toddlers,for babies,yippee toys
marble maze,teach colors,learn colors,marbles,marble maze run,learn with marbles,marble run,marble maze for kids,toddler learning videos,preschool learning video,children learning video,children learning videos,toddler learning video,funeggvideos,learning videos for toddlers,toddler learning,toddler videos,children videos,learning videos,learning videos for kids,learning videos for children,learning videos for preschool,preschool toys,toys,kids
kinder,chocolate eggs,surprise toys,finding dory,zootopia,frozen,barbie,toy
Kitchen,Appliance,Toys,for Kids,Coffee,Maker,Machine,Kettle,Just Like Home,children,toddlers,playing,pretend,play set
Unboxing Toys,funny videos,toys for kids,toys,kids songs,baby learn,learn colors,unboxing toys,Learning Numbers,Stop Motion,Surprise Eggs,50 Surprise Eggs
rikki tikki tavi full movie,adventure movie,animal movies,story of a jungle,children's motivational movies,

Lego Movie 2,Lego Movie,Emmet,Dinosaur,Thricycle,Lego,Lego Movie 2 The Second Part,Rex Dangervest,Lucy,Time Machine,Duplo,Legos,Duplos,Duplo Lego,Lego Duplo,Dinosaur Attack,T-Rex,Time Travel,Lego Movie 2 Parody,Lego Movie 2 Rocket House,batman vs superman,batman toys,batman arkham city,batman beyond,batman arkham knight,dinosaur,kids toys,toysreviewtoys,toys review toys,toys review,sandaroo,sandaroo family,review toys
[]
wendy,pretend play,wendy pretend play,kids pretend play,elmo,giant elmo,elmo costume,hide and seek,play,kids play,toys play,toys and colors,toy,toys,children stories,fun,funny,cute,kids,family friendly,video for kids,family fun,children playing,hiding
Chocolate (Industry),Chocolate Fountain,Play Go,Desserts,kid's toys,Toy (Industry),cake,Dessert (Type Of Dish),Chocolate Bar (Dish),chocolate egg,Marshmallow (Breakfast Cereal Flavor),Marshmallow Creme (Invention),toys,Toy Kingdom (Business Operation),Milk,play,surprise,surprise egg,Food (Industry),foods
spider,redback sp

Parenting tips,Editor's picks,Big News,Parents,The TODAY Show,TODAY Show,TODAY,NBC,NBC News,Celebrity Interviews,Lifestyle,TODAY Show Interview,TODAY show news,news,Ryan ToysReview,ToysReview,YouTube channel,misleading young viewers,Ryan ToysReview Accused,ryan toysreview Deceiving Kids,Ryan ToysReview paid content,ryans toy review,ryan toy review,ryan toys review,toys review paid content,Morgan Radford,the latest on today,youtuber ryan toysreview
toys,kids toys,hot wheels,car wash,kids car wash,inflatable toys,toys for kids,children,kids,babies,Ryan ToysReview,Ryan,Emma,Kate,Ryan's Family Review,Step2,Cozy Coupe,children activties,water toys,kids video,family,family fun,Mr Bubbles,Foam Soap,washing toy cars,Gus The Gummy Gator,Cars,kids Cars
FASTFOODTOYREVIEWS,cartoon,animated,promotion,mcdonalds,burger king,wendys,arbys,kids,toys,children,play set,collection,mini figure,full set,drive thru,grimace,ronald,birdie,hamburglar,caddy,unboxing,playskool,school supplies,crayola crayon,play d

mickey mouse,minni mouse,Disney Channel,Mickey Mouse Clubhouse,Disney Junior,Mickey Mouse Toys,Minnie Mouse Toys,Giant Egg Surprises,Surprise eggs,surprise eggs opening,surprise toys,Ryan ToysReview,Biggest surprise egg ever,toy kinder surprise,Toy Surprise,toys collector,giant disney eggs,disney toys,mickey mouse video,disney junior videos,Mickey Mouse Full,mickey mouse power wheels,ride on kids car,Kids Video
colorful blocks,giant blocks for kids,kids play with blocks,rainbow toys review,toys for children,kids videos,huge blocks,colorful toys
cute zoo animals,cute animals,zoo animals,fun zoo,Kids zoo,cute toys,zoo,toy review,toys,zoo video,animal video,happy,kids toys,hippopotamus,kids,fun video,play,animals,toy animals,cute,cute zoo,fun toys,wildlife,grizzly bear,camel,fox,new,wild,wild animals,funny,Animales salvajes,Animali selvaggi,grizzly,monkey,hippo,3+,children,toddlers,videos,play time,game,zoo animal video,animal toys,animal,fun ending,toy collection
stay at home,kids,childr

cars for babies,cars for kids,kids cars,car toys,Toys,toys for kids,for kids,video for kids,toy vehicles,cars toys,hot wheels cars,toys,toy cars,toys play,cars for children,toy,car toys review,kids cars review,videos for kids
Review kids toys,Animals,Toy elephant,toy goat,toy rhinoceros,for kids,for kid,review toys,toys review,toy for kids,surprise,surprises,surprise toys,learn wild animals,learn,toy,toys,kids,children,video for children,for baby,review,new,2016,FunnyBigBabyTV
FASTFOODTOYREVIEWS,cartoon,animated,promotion,mcdonalds,burger king,wendys,arbys,kids,toys,children,play set,collection,mini figure,full set,drive thru,bowser,princess peach,disney,super nintendo,ds,luigi,mushroom,yoshi,wii u,voltron,despicable me 2 3,minions,toy story 3 4,hulk,thor,spider-man,batman,tmnt,comic book character,lego,playskool,tv,show,episode,groot,guardians of the galaxy,wonder woman,marvel,dc,superman
paw patrol,surprise eggs,kinder surprise,video for kids,toys for kids,toys review,мультик для дет

In [219]:
corpus = list(res_df['tags'])

In [220]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [221]:
print(X.shape)

(1234, 4503)


In [222]:
type(X.toarray())

numpy.ndarray

In [223]:
len(vectorizer.get_feature_names())

4503

In [224]:
new_x = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names(), index = res_df['video'])

In [225]:
new_x.head()

,10,100,1000pcs,101,1080p,12,12v,150,19,1978,...,타요,탱크,토이리뷰,토이즈,토이팜,토이포키즈,트럭놀이,포크레인,플레이,화장품
video,,,,,,,,,,,,,,,,,,,,,
GJI9BsrJipY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7HVqxq2VuYU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.331959,0.0,0.0,0.0,0.0
RT6tMatSnlA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
BbZlJPT9jYY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
_cjFJI-Nhtc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [226]:
new_x.sum().sort_values(ascending = False)[:40]

kids          118.299010
toys          109.415526
for            66.243710
play           58.965600
toy            57.971031
pretend        43.020451
surprise       39.449579
ryan           38.830655
family         35.745260
lego           34.575931
children       33.656117
video          29.923708
fun            29.742307
videos         29.289537
review         27.567044
cars           27.412405
game           27.346268
egg            22.473786
car            21.720066
learning       20.748409
and            20.486612
unboxing       20.023525
eggs           19.668275
channel        19.525382
baby           19.113493
colors         18.632336
kid            18.338495
playing        18.316523
food           18.093812
story          18.020825
doll           17.474604
build          17.169915
best           16.474801
funny          16.145933
friendly       16.109363
disney         16.030722
new            15.914397
games          15.819343
toysreview     15.584853
learn          15.507468


In [125]:
final_ls = []
for ls in list(res_df['tags'].str.split(',')):
    final_ls.extend(ls)

In [126]:
from collections import Counter
counter_1 = Counter(final_ls)

In [124]:
final_ls

['mcdonalds',
 'mcdonalds toys',
 'pretend play',
 'pretend play mcdonalds',
 'Ryan ToysReview',
 "Ryan's Toy",
 'Ryan Toys',
 'toys',
 'kids toys',
 'kitchen toys',
 'cooking toys',
 'pretend food',
 'pretend food toys',
 'kitchen playset',
 'hamburger toys',
 'french fries toys',
 'for kids',
 'kids playing',
 'kids pretend pkay',
 'food',
 'children',
 'food playset',
 'kids playset',
 'cash registers playset',
 'cash register',
 'Ryan ToysReview',
 'PJ Masks',
 'PJ Masks Giant Egg',
 'PJ Masks IRL',
 'PJ Masks giant egg surprise',
 'Pj Masks Toys videos',
 'toys',
 'kids toys',
 'DIsney Toys',
 'PJ Masks Superhero',
 'PJ Masks Costumes',
 'Headquarters Playset',
 'Catboy',
 'Gekko',
 'Owlette',
 'Romeo',
 'IRL Superheroes',
 'New pj masks',
 'surprise eggs',
 'egg surprise',
 'toys unboxing',
 'kids video',
 'romeo',
 "Ryan's Family Review",
 'eggs',
 'kids',
 'children',
 'Pj Masks Toys',
 "Ryan's Family Review",
 "Ryan's Family",
 'Ryan ToysReview',
 "Ryan's Toy Review",
 'Ryan',

In [127]:
counter_1.most_common(100)

[('Ryan ToysReview', 18),
 ('kids toys', 15),
 ('pretend play', 14),
 ('children', 14),
 ('kids', 14),
 ('toys', 11),
 ('for kids', 11),
 ('toy', 11),
 ('kids video', 8),
 ('toys for kids', 8),
 ("Ryan's Family Review", 7),
 ('Ryan', 7),
 ('unboxing', 7),
 ('lego', 7),
 ('play', 7),
 ('funny', 6),
 ('Toys', 6),
 ('pretend', 6),
 ('fun', 6),
 ('cute', 6),
 ('family', 5),
 ('family fun', 5),
 ('doll', 5),
 ('review', 5),
 ('roleplay', 5),
 ('demonstrate', 5),
 ('create', 5),
 ('make', 5),
 ('demo', 5),
 ('toddler', 5),
 ('preschool', 5),
 ('boy', 5),
 ('girl', 5),
 ('teen', 5),
 ('tween', 5),
 ('great', 5),
 ('best', 5),
 ('good', 5),
 ('build', 5),
 ('friendly', 5),
 ('safe', 5),
 ('entertainment', 5),
 ('lev', 5),
 ("Ryan's Toy", 4),
 ("Ryan's Family", 4),
 ('Family Fun', 4),
 ('surprise toys', 4),
 ('spider-man', 4),
 ('play doh', 4),
 ("Ryan's World", 4),
 ('costume', 4),
 ('irl', 4),
 ('Kids', 4),
 ('Ryantoysreview', 4),
 ('education', 4),
 ('new', 4),
 ('channel', 4),
 ('story', 4)

In [176]:
def extract_video_from_json(directory):

    videos = []
    for entry in os.scandir(directory):
        if entry.path.endswith(".json") and entry.is_file():
            items = [json.loads(line) for line in open(entry, 'r')]
            # items = json.load(open(entry, 'r'))
            new_videos = [item['id']['videoId'] for item in items['items']]
            videos.extend(new_videos)

    return videos

In [182]:
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()
        
        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            raise
        yield obj

In [183]:
import re
from json import JSONDecoder, JSONDecodeError

NOT_WHITESPACE = re.compile(r"[^\s]")


def stream_json(file_obj, buf_size=1024, decoder=JSONDecoder()):
    buf = ""
    ex = None
    while True:
        block = file_obj.read(buf_size)
        if not block:
            break
        buf += block
        pos = 0
        while True:
            match = NOT_WHITESPACE.search(buf, pos)
            if not match:
                break
            pos = match.start()
            try:
                obj, pos = decoder.raw_decode(buf, pos)
            except JSONDecodeError as e:
                ex = e
                break
            else:
                ex = None
                yield obj
        buf = buf[pos:]
    if ex is not None:
        raise ex

In [192]:
ids = stream_json(open('/Users/QiaoLiu1/kids_tube/collected_link_json/youtube_request_CN4CEAA.json',
                                           'r'), buf_size=1024*25)

In [193]:
id_1 = []

In [194]:
for idd in ids:
    id_1.append(idd)

In [203]:
id_1[1]['items'][0]['id']['videoId']

'MILPdXintcw'

In [204]:
id_1[0]['items'][0]['id']['videoId']

'WRsGFeZjcho'

In [ ]:
items = decode_stacked()